<a href="https://colab.research.google.com/github/AhlemKaabi/holbertonschool-machine_learning/blob/main/supervised_learning/0x09-transfer_learning/0_transfer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
"""
    Transfer Knowledge
"""
import tensorflow.keras as K
from keras import layers
from keras.models import Model
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D, Activation, Flatten
from keras.layers import Dense, Dropout
from keras.applications.inception_v3 import InceptionV3
from keras.utils.np_utils import to_categorical

from keras.layers import Lambda, Input

import matplotlib.pyplot as plt


# Data Preprocessing

In [ ]:
def preprocess_data(X, Y):
    """
    Method:
        pre-processes the data for your model.
    """
    X_p = K.applications.inception_v3.preprocess_input(X)
    Y_p = K.utils.to_categorical(Y, 10)

    return X_p, Y_p

In [ ]:
(x_train, y_train), (x_test, y_test) = K.datasets.cifar10.load_data()

170508288/170498071 [==============================] - 13s 0us/step


In [ ]:
x_train.shape

In [ ]:
x_train[0] # cheking one image

In [ ]:
plt.imshow(x_train[120])

In [ ]:
y_train.shape

In [ ]:
y_train[0]

In [ ]:
X_train_p, Y_train_p = preprocess_data(x_train, y_train)
X_test_p, Y_test_p = preprocess_data(x_test, y_test)

In [ ]:
X_train_p.shape

In [ ]:
X_train_p[0]

In [ ]:
Y_train_p.shape

In [ ]:
Y_train_p[0]

# Transfer Learning

In [ ]:
base_inception = InceptionV3(include_top=False, input_shape=(299, 299, 3))

87924736/87910968 [==============================] - 3s 0us/step


In [ ]:
# base_inception.summary()

In [ ]:
print(base_inception.input.shape)
print(base_inception.output.shape)

(None, 299, 299, 3)
(None, 8, 8, 2048)


In [ ]:
input_layer = Input(shape=(32, 32, 3))

resizing_layer = Lambda(lambda image: K.preprocessing.image.smart_resize(image, (299, 299)))(input_layer)
# print(resizing_layer.shape)

inception_layers = base_inception(resizing_layer, training=False)

glob_pooling = GlobalAveragePooling2D()(inception_layers)

layer_i = Dense(500, activation='relu')(glob_pooling)

dropout_layer = Dropout(0.3)(layer_i)

output_layer =  Dense(10, activation='softmax')(dropout_layer)

model = Model(inputs=input_layer, outputs=output_layer)

In [ ]:
base_inception.trainable = False

optimizer = K.optimizers.Adam(0.001)

In [ ]:
model.compile(optimizer=optimizer,
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
es = K.callbacks.EarlyStopping(monitor='val_loss', verbose=1)
# By default, mode is set to ‘auto‘ and knows that you want to minimize loss or maximize accuracy.

In [ ]:
history = model.fit(
        X_train_p,
        Y_train_p,
        validation_data=(X_test_p, Y_test_p),
        batch_size=300,
        epochs=15,
        verbose=1,
        callbacks=[es])
# changed the epoches to 15


Epoch 1/15
167/167 [==============================] - 229s 1s/step - loss: 0.5672 - accuracy: 0.8119 - val_loss: 0.4056 - val_accuracy: 0.8603
Epoch 2/15
167/167 [==============================] - 201s 1s/step - loss: 0.3905 - accuracy: 0.8660 - val_loss: 0.4037 - val_accuracy: 0.8616
Epoch 2: early stopping


In [ ]:
results = model.evaluate(X_test_p, Y_test_p)


313/313 [==============================] - 38s 116ms/step - loss: 0.4037 - accuracy: 0.8616


In [ ]:
print(results)

[0.39033597707748413, 0.8687000274658203]


In [ ]:
from tensorflow.keras.utils import plot_model
plot_model(base_inception, to_file='model.png')

In [ ]:
for i, layer in enumerate(base_inception.layers):
  print(i, layer.name)

In [ ]:
# for layer in base_inception.layers[:195]:
#   layer.trainable=False

# for layer in base_inception.layers[195:]:
#   layer.trainable=True
# --> loss: 0.3791 - accuracy: 0.8719
for layer in base_inception.layers[:164]:
  layer.trainable=False

for layer in base_inception.layers[164:]:
  layer.trainable=True

In [ ]:
my_optimizer = K.optimizers.Adam(1e-5)


model.compile(optimizer=my_optimizer,
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
es = K.callbacks.EarlyStopping(monitor='val_loss', mode='min', verbose=1)
history = model.fit(
        X_train_p,
        Y_train_p,
        validation_data=(X_test_p, Y_test_p),
        batch_size=300,
        epochs=10,
        verbose=1,
        callbacks=[es])

Epoch 1/10
167/167 [==============================] - 200s 1s/step - loss: 0.3344 - accuracy: 0.8852 - val_loss: 0.3620 - val_accuracy: 0.8768
Epoch 2/10
167/167 [==============================] - 202s 1s/step - loss: 0.3217 - accuracy: 0.8908 - val_loss: 0.3598 - val_accuracy: 0.8768
Epoch 2: early stopping


In [ ]:
results = model.evaluate(X_test_p, Y_test_p)

313/313 [==============================] - 84s 268ms/step - loss: 0.3757 - accuracy: 0.8724
